<img src=https://i.ytimg.com/vi/knxlmCVFAZI/maxresdefault.jpg>

In [1]:
!pip3 install tqdm


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
from twilio.rest import Client
from twilio_config import *
import time

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json


import pandas as pd
import requests
from bs4  import BeautifulSoup
from tqdm import tqdm

from datetime import datetime


# Armado de la URL

In [3]:
query = 'Cochabamba'
api_key = 'f3cf65ff7ae64bccad9210657220807'

url_clima = 'http://api.weatherapi.com/v1/forecast.json?key='+api_key+'&q='+query+'&days=1&aqi=no&alerts=no'
url_clima

'http://api.weatherapi.com/v1/forecast.json?key=f3cf65ff7ae64bccad9210657220807&q=Gaithersburg&days=1&aqi=no&alerts=no'

In [4]:
response = requests.get(url_clima).json()

In [5]:
response

{'location': {'name': 'Gaithersburg',
  'region': 'Maryland',
  'country': 'United States of America',
  'lat': 39.14,
  'lon': -77.2,
  'tz_id': 'America/New_York',
  'localtime_epoch': 1714193219,
  'localtime': '2024-04-27 0:46'},
 'current': {'last_updated_epoch': 1714193100,
  'last_updated': '2024-04-27 00:45',
  'temp_c': 11.1,
  'temp_f': 52.0,
  'is_day': 0,
  'condition': {'text': 'Overcast',
   'icon': '//cdn.weatherapi.com/weather/64x64/night/122.png',
   'code': 1009},
  'wind_mph': 11.9,
  'wind_kph': 19.1,
  'wind_degree': 130,
  'wind_dir': 'SE',
  'pressure_mb': 1032.0,
  'pressure_in': 30.46,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'humidity': 46,
  'cloud': 100,
  'feelslike_c': 9.5,
  'feelslike_f': 49.1,
  'vis_km': 16.0,
  'vis_miles': 9.0,
  'uv': 1.0,
  'gust_mph': 12.8,
  'gust_kph': 20.6},
 'forecast': {'forecastday': [{'date': '2024-04-27',
    'date_epoch': 1714176000,
    'day': {'maxtemp_c': 11.1,
     'maxtemp_f': 51.9,
     'mintemp_c': 7.3,
     'mint

In [6]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [7]:
response['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [8]:
len(response['forecast']['forecastday'][0]['hour'])

24

In [9]:
response['forecast']['forecastday'][0]['hour'][0]

{'time_epoch': 1714190400,
 'time': '2024-04-27 00:00',
 'temp_c': 8.5,
 'temp_f': 47.4,
 'is_day': 0,
 'condition': {'text': 'Partly Cloudy ',
  'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png',
  'code': 1003},
 'wind_mph': 9.6,
 'wind_kph': 15.5,
 'wind_degree': 135,
 'wind_dir': 'SE',
 'pressure_mb': 1032.0,
 'pressure_in': 30.47,
 'precip_mm': 0.0,
 'precip_in': 0.0,
 'snow_cm': 0.0,
 'humidity': 61,
 'cloud': 32,
 'feelslike_c': 6.2,
 'feelslike_f': 43.2,
 'windchill_c': 6.2,
 'windchill_f': 43.2,
 'heatindex_c': 8.5,
 'heatindex_f': 47.4,
 'dewpoint_c': 1.3,
 'dewpoint_f': 34.4,
 'will_it_rain': 0,
 'chance_of_rain': 0,
 'will_it_snow': 0,
 'chance_of_snow': 0,
 'vis_km': 10.0,
 'vis_miles': 6.0,
 'gust_mph': 15.0,
 'gust_kph': 24.1,
 'uv': 1.0}

In [10]:
response['forecast']['forecastday'][0]['hour'][0]['time'].split()[0]

'2024-04-27'

In [11]:
response['forecast']['forecastday'][0]['hour'][1]['time'].split()[1].split(':')[0]

'01'

In [12]:
response['forecast']['forecastday'][0]['hour'][0]['will_it_rain']

0

In [13]:
response['forecast']['forecastday'][0]['hour'][2]['chance_of_rain']

0

# Dataframe

In [14]:
def get_forecast(response,i):
    
    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hora = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0])
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    tempe = float(response['forecast']['forecastday'][0]['hour'][i]['temp_c'])
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    prob_rain = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']
    
    return fecha,hora,condicion,tempe,rain,prob_rain

In [15]:
datos = []

for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])),colour = 'green'):
    
    datos.append(get_forecast(response,i))
    

100%|██████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<?, ?it/s]


In [16]:
datos[0]

('2024-04-27', 0, 'Partly Cloudy ', 8.5, 0, 0)

In [17]:
col = ['Fecha','Hora','Condicion','Temperatura','Lluvia','prob_lluvia']
df = pd.DataFrame(datos,columns=col)
df = df.sort_values(by = 'Hora',ascending = True)
df

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
0,2024-04-27,0,Partly Cloudy,8.5,0,0
1,2024-04-27,1,Overcast,11.1,0,0
2,2024-04-27,2,Partly Cloudy,7.9,0,0
3,2024-04-27,3,Cloudy,8.0,0,0
4,2024-04-27,4,Overcast,8.4,0,0
5,2024-04-27,5,Overcast,8.8,0,0
6,2024-04-27,6,Partly Cloudy,8.9,0,0
7,2024-04-27,7,Cloudy,9.2,0,0
8,2024-04-27,8,Cloudy,9.8,0,0
9,2024-04-27,9,Overcast,10.2,0,0


In [18]:
# data_cli = {
#     'Fecha':datos[0][0],
#     'Hora':datos[1][],
#     'Tiempo':condicion,
#     'Temperatura':tempe,
#     'LLoverá':rain,
#     'prob_lluvia':prob_rain
    
# }

# df = pd.DataFrame(data_cli)
# df = df.sort_values(by = 'Hora',ascending = True)

In [19]:
df[df['Lluvia']==1]

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
11,2024-04-27,11,Patchy rain nearby,10.5,1,100
12,2024-04-27,12,Patchy rain nearby,10.3,1,100
13,2024-04-27,13,Light drizzle,10.1,1,100
14,2024-04-27,14,Light drizzle,9.9,1,100
15,2024-04-27,15,Light drizzle,9.7,1,100
16,2024-04-27,16,Light drizzle,10.1,1,100
17,2024-04-27,17,Light drizzle,9.9,1,100
18,2024-04-27,18,Light drizzle,10.0,1,100


In [20]:
df_rain =  df[(df['Lluvia']==0
              ) & (df['Hora']>6) & (df['Hora']< 22)]
df_rain = df_rain[['Hora','Condicion']]
df_rain.set_index('Hora', inplace = True)

In [21]:
df['Fecha'][0]

'2024-04-27'

In [22]:
df_rain

,Condicion
Hora,
7,Cloudy
8,Cloudy
9,Overcast
10,Overcast
19,Cloudy
20,Cloudy
21,Partly Cloudy


In [23]:
'\nHola! \n\n\n Paola, el pronostico del tiempo hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain)

'\nHola! \n\n\n Paola, el pronostico del tiempo hoy 2024-04-27 en Gaithersburg es : \n\n\n            Condicion\nHora                \n7            Cloudy \n8            Cloudy \n9          Overcast \n10         Overcast \n19           Cloudy \n20           Cloudy \n21    Partly Cloudy '

In [24]:
PHONE_NUMBER

'+15065604561'

# Mensaje SMS desde Twilio

In [41]:
time.sleep(2)
account_sid = ACCOUNT_SID
auth_token = AUTH_TOKEN

client = Client(account_sid, auth_token)

message = client.messages \
                .create(
                     body='\nHola! \n\n\n El pronostico de lluvia hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain),
                     from_=PHONE_NUMBER,
                     to=''
                 )

print('Mensaje Enviado ' + message.sid)

TwilioRestException: HTTP 400 error: Unable to create record: Invalid 'To' Phone Number: (BO)+15916944XXXX

# Challenge 

* Extrae el valor del dolar en tu país y el top 10 de criptomonedas con su respectiva valoración
* Ahora envia un mensaje diarío a tu Whatsapp usando Twilio

**hint 💡** Investiga que API's gratuitas existen para consultar estos datos



<img src="WhatsApp Image 2022-09-13 at 9.12.18 AM.jpeg" width="200" height="200" />